The decorator is a callable that takes another function as argument. The decorator helps to transform the function as follows.

In [ ]:
@decorator
def target():
    print('running target()')

# has the same effect as

def target():
    print('running target')
    
target = decorator(target)

The first crucial fact about decorators is that they have the power to replace the decorated function with a different one. Second crucial fact is that they are exectuted immediately when a module is loaded.

In [2]:
registry = []

def register(func):
    print('running register (%s)'%func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('register -> ', registry)
    f1()
    f2()
    f3()


running register (<function f1 at 0x7f9564048050>)
running register (<function f2 at 0x7f95640b85f0>)


In [3]:

main()

running main()
register ->  [<function f1 at 0x7f9564048050>, <function f2 at 0x7f95640b85f0>]
running f1()
running f2()
running f3()


This emphasis the point that function decorators are executed as soon as the module is imported but the imported functions run only when they are explicitly called. This is what you call `importtime` and `runtime`

## Decorator-Enhanced Strategy Pattern

Decorators offer a way of implementing the `bestprom` functionality from the prev chapter. Here we can use decorators to register each promo code and that way minmise code reuse.

In [5]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

Now in the above examples we saw cases the decorators send back the same function, but most of the decorators do change the function. This is done by defining a inner function and returning that. To understand that better lets look at Closures and varialbe scope rules.

## Variable Scope Rules

In [1]:
def f1(a):
    print(a)
    print(b)
    
f1(4)

4


NameError: name 'b' is not defined

In [2]:
# expect!
# but if we define a global variable b 

b = 10
f1(4)

4
10


In [3]:
def f2(a):
    print(a)
    print(b)
    b = 5
    
f2(10)

10


UnboundLocalError: local variable 'b' referenced before assignment

Why this happens?

when Python compiles the body of the function, it decides that b is a local
variable because it is assigned within the function. The generated bytecode reflects this
decision and will try to fetch b from the local environment. Later, when the call f2(3)
is made, the body of f2 fetches and prints the value of the local variable a , but when
trying to fetch the value of local variable b it discovers that b is unbound.

In [4]:
def f3(a):
    # use the global b
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
10


In [5]:
b

9

In [8]:
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [9]:
dis(f2)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  4          16 LOAD_CONST               1 (5)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## Closues

closue is a function with an extended scope that encompasses nonglobal variables referenced in the body fo the function but not defined there. 

Now to show this in practice lets build a function `avg()` that calculates the average of all the numbers that are called using it.

In [14]:
# a higher order function for this functionality
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [15]:
avg = make_averager()
avg(10)

10.0

In [16]:
avg(11)

10.5

In [17]:
avg(12)

11.0

look at `series` variable. Its accessed in `averager()` function where its is no longer a local variable, but `averager` is still able to call `series.append()`. 

In this case `series` is what we technically call a free variable. It is a variable that is not bound in the local scope.

Lets inspect the function created by make_averager

In [20]:
avg.__code__.co_varnames

('new_value', 'total')

In [21]:
avg.__code__.co_freevars

('series',)

The binding for series is kept in the `__closure__` attribute of the return function avg. Each item in `avg.__closure__` corresponds to a name in `avg.__code__.co_freevars`. These items are `cells`, and they have an attribute called `cell_contents` where the actual values can be found.

In [22]:
avg.__code__.co_freevars

('series',)

In [23]:
avg.__closure__

(<cell at 0x7f3a548da890: list object at 0x7f3a548bfe10>,)

In [24]:
avg.__closure__[0].cell_contents

[10, 11, 12]

To summarize: a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

*Note* that the only situation in which a function may need to deal with external variables
that are nonglobal is when it is nested in another function.

## nonlocal Declaration
Our implementation of `make_average` is not opimal. A better way would be to problably store the total and the number of values. 

Now lets look at a broken implementation of the same.

In [2]:
def make_average():
    total = 0
    count = 0
    
    def average(new_value):
        total += new_value
        count += 1
        return total/count
    
    return average

avg = make_average()
avg(10)

UnboundLocalError: local variable 'total' referenced before assignment

Here if you notice, inside the `average()` function the statement is count += 1 where count is a number or an immutable type. But this statement makes it a local variable.

We didn't have this previously beacause it was a list and mutable, but for immutable objects which we can only read this is not possible.

To work arond this, the `nonlocal` declaration was introduced.

In [6]:
def make_averager():
    total = 0
    count = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

avg = make_averager()
avg(10)

10.0

## simple Decorator

In [12]:
# simple decorator to output the running time of functions
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r'%(elapsed, name, arg_str, result))
        return result
    return clocked

In [18]:
# clockdeco_demo.py
import time

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12308057s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000084s] factorial(1) -> 1
[0.00003076s] factorial(2) -> 2
[0.00005233s] factorial(3) -> 6
[0.00007870s] factorial(4) -> 24
[0.00010557s] factorial(5) -> 120
[0.00013350s] factorial(6) -> 720
6! = 720


In [14]:
factorial.__name__

'clocked'

As you can see the factorial function is changed an points to the function returned by the doecorator. The function in the decorator is doing the work.

Our decorator has a few issues, it doesnot handle keyword arguments and masks the `__name__` and `__doc__`. We hence use functools.wraps decorator

In [17]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r'%(elapsed, name, arg_str, result))
        return result
    return clocked

In [20]:
factorial.__name__

'factorial'

## Decorators in the Standard Lib
Python has three build in decorators, `property`, `classmethod`, `staticmethod`. Other common ones are from `functools`. They are `lru_cache` and `singledispactch`

### `lru_cache`
This implements memoization, optimization technique that works by saving the results of previous invocations of expensive functions. Lets see it in action by applying it on a recursive fibnacci finder.

In [21]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    else :
        return fibonacci(n-1) + fibonacci(n-2)

In [22]:
print(fibonacci(6))

[0.00000163s] fibonacci(1) -> 1
[0.00000360s] fibonacci(0) -> 0
[0.00078144s] fibonacci(2) -> 1
[0.00000222s] fibonacci(1) -> 1
[0.00098398s] fibonacci(3) -> 2
[0.00000149s] fibonacci(1) -> 1
[0.00000183s] fibonacci(0) -> 0
[0.00036715s] fibonacci(2) -> 1
[0.01037849s] fibonacci(4) -> 3
[0.00000280s] fibonacci(1) -> 1
[0.00000298s] fibonacci(0) -> 0
[0.00460281s] fibonacci(2) -> 1
[0.00000432s] fibonacci(1) -> 1
[0.00512743s] fibonacci(3) -> 2
[0.01681365s] fibonacci(5) -> 5
[0.00000235s] fibonacci(1) -> 1
[0.00000286s] fibonacci(0) -> 0
[0.00029033s] fibonacci(2) -> 1
[0.00000251s] fibonacci(1) -> 1
[0.00055052s] fibonacci(3) -> 2
[0.00000216s] fibonacci(1) -> 1
[0.00000257s] fibonacci(0) -> 0
[0.00025737s] fibonacci(2) -> 1
[0.00108055s] fibonacci(4) -> 3
[0.01815774s] fibonacci(6) -> 8
8


As you can see there is a lot of wasted cycles.

In [24]:
import functools 

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    else :
        return fibonacci(n-1) + fibonacci(n-2)
    
fibonacci(6)

[0.00000093s] fibonacci(1) -> 1
[0.00000208s] fibonacci(0) -> 0
[0.00046714s] fibonacci(2) -> 1
[0.00052559s] fibonacci(3) -> 2
[0.00056689s] fibonacci(4) -> 3
[0.00060813s] fibonacci(5) -> 5
[0.00065066s] fibonacci(6) -> 8


8

other than making silly recursinve functions faster, they are also super helpfull in applications that fetches info from the web

They have additional arguments

`lru_cache(maxsize=128, typed=True)`

### `singledispatch`

In [27]:
# generate HTML displays of various objects
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [28]:
htmlize({1, 2, 4})

'<pre>{1, 2, 4}</pre>'

What we want to build is a `htmlize` that displays according to the data that is passed. Since we have no function overloading in Python we cannot create different function for each signature. What we usally do is create a dispatch function with many `if/elif/else` statements to handle it.

But another way is to use `functools.singledispatch`. If a function is decorated with this, it becomes a `generic function`.  See the example to know how this is implemented.

In [32]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [33]:
htmlize('hai')

'<p>hai</p>'

In [34]:
htmlize('hai\nbye')

'<p>hai<br>\nbye</p>'

**tip:** When possible, register the specialized functions to handle ABCs (abstract classes) such as `numbers.Integral` and `abc.MutableSequence` instead of concrete implementations like `int` and `list` . This allows your code to support a greater variety of compatible types. For example, a Python extension can provide alternatives to the `int` type with fixed bit lengths as subclasses of `numbers.Integral`.

A notable feature is that additional mechanisms to handle some datatypes can be registered from anywhere in the system, in any module.